<h1><center>DSA 4212: Year 2019-2020</center></h1>
<h3><center> Assignment 1 (Deadline: 27 March 2020 at 23:59) </center></h3>
<h3><center> To Be submitted on the lumiNUS )</center></h3>
<h2><center> Group Number: ???? </center></h2>
<h2><center> Group Member 1: Joyce Lim Li Jie, Student ID </center></h2>
<h2><center> Group Member 2: Ryan Teo Jin Chuan, A0164337A </center></h2>
<h2><center> Group Member 3: Wong Jia Hwee, Student ID </center></h2>



In [212]:
%matplotlib inline
from jax.config import config
config.update("jax_enable_x64", True) 

import jax
import jax.numpy as np

import pylab as plt
import imageio
import os
import numpy as onp
from skimage.transform import rescale, resize, downscale_local_mean
import pandas as pd
import time
import scipy

# Data Dowload
1. Download the "celeba_small.zip" file available on lumiNUS.
This is a 146Mo large zip-file containing 20K face images.
2. Download the attribute file "celeba.csv" available on lumiNUS.
3. Unzip the file "celeba_small.zip" in the directory of your choice. (Data = 175 Mo when uncompressed). 

# Brief Data exploration

In [ ]:
#load attributes csv file
attribute = pd.read_csv("celeba.csv")

In [ ]:
#"attribute" is a dictionary containing several attributes for each image
attribute.keys()

In [ ]:
#let us list all the files in the image directory
path = "img_celeba_small"
all_img = [f for f in os.listdir(path) 
                 if os.path.isfile(os.path.join(path, f)) 
                 and f.endswith(".jpg")]
nb_img = len(all_img)
print("Number of images:", nb_img)

#let us keep only the relevant attributes
attribute = attribute[:20000]

In [ ]:
#let us display the first 20 images
plt.figure(figsize=(10,2))
for k in range(20):
    #load image
    im = imageio.imread(os.path.join(path, all_img[k])).astype(float)
    #resize to 100x100 for display
    im = resize(im, (100,100) )
    #scale pixel intensity to [0,1] by divising by 255 and display
    plt.subplot(2,10,k+1)
    plt.imshow(im/255.)
    plt.axis("off")
    
    is_male = attribute["Male"][k]
    if is_male == 1:
        plt.title("Male")
    else:
        plt.title("Female")

In [ ]:
#let us display the first 20 images in black and white
plt.figure(figsize=(10,2))
for k in range(20):
    #load image
    im = imageio.imread(os.path.join(path, all_img[k])).astype(float)
    #resize to 100x100
    im = resize(im, (100,100) )
    #transform to black and white by averaging the 3 color channels
    im = onp.mean(im, axis=2)
    #scale pixel intensity to [0,1] by divising by 255 and display
    plt.subplot(2,10,k+1)
    plt.imshow(im/255., cmap="gray") # grayscale
    plt.axis("off")

In [ ]:
#let us load the first 15000 images
n_img = 15000
img_array = onp.zeros((n_img,100,100))
for k in range(n_img):
    im = imageio.imread(os.path.join(path, all_img[k])).astype(float)
    im = resize(im, (100,100))
    im = onp.mean(im, axis=2)
    img_array[k,:,:] = im

In [ ]:
#let us plot a few "average" faces
plt.figure(figsize=(10,10))

plt.subplot(2,5,1)
is_young = attribute["Young"][:n_img]==1
plt.imshow(onp.mean(img_array[is_young,:,:], axis=0), cmap="gray")
plt.axis("off")
plt.title("Avg(Young)")

plt.subplot(2,5,2)
is_male = attribute["Male"][:n_img]==1
plt.imshow(onp.mean(img_array[is_male,:,:], axis=0), cmap="gray")
plt.axis("off")
plt.title("Avg(Male)")


plt.subplot(2,5,3)
is_Goatee = attribute["Goatee"][:n_img]==1
plt.imshow(onp.mean(img_array[is_Goatee,:,:], axis=0), cmap="gray")
plt.axis("off")
plt.title("Avg(Goatee)")


plt.subplot(2,5,4)
is_High_Cheekbones = attribute["High_Cheekbones"][:n_img]==1
plt.imshow(onp.mean(img_array[is_High_Cheekbones,:,:], axis=0), cmap="gray")
plt.axis("off")
plt.title("Avg(High_Cheek)")


plt.subplot(2,5,5)
is_Low_Cheekbones = attribute["High_Cheekbones"][:n_img]==-1
plt.imshow(onp.mean(img_array[is_Low_Cheekbones,:,:], axis=0), cmap="gray")
plt.axis("off")
plt.title("Avg(Low_Cheek)")

In [ ]:
print("Proportion of Young people in the dataset:", np.mean(attribute["Young"]==1) )

# Assignment

Your assignment consists in building an algorithm that can automatically tell whether an image corresponds to a young person or not. 

1. You are only allowed to use a logistic regression model (possibly with regularization). 
2. You can only use the first 15,000 images to train you model. The accuracy of your model will be evaluated on the last 5,000 images.
3. You are allowed to use whatever optimization algorithm you think is most efficient.
4. You are allowed to do whatever pre-processing you deem appropriate.
5. You will report the accuracy (i.e. th percentage of correctly classified) on the test dataset (i.e. the last 5,000 images). 
6. You will as well report the Area Under the Curve (AUC) of your classifier on the test dataset.

### Some remarks:
1. You can work with colored, or grayscale images.
2. You can rescale the images to whatever resolution you think is best/efficient
3. It is usually a good idea to rescale the pixel intensity in between 0 and 1
4. If you fit a logistic regression in dimension $D$, with each coordinate roughly in between 0 and 1, it is usually a good idea to start with a random $\beta$ with mean 0 (or 0.5), and standard deviation $1/\sqrt{D}$ for each coordinate.
5. It may be a good (or bad?) idea to increase/lower the contrast of the image. 
6. It may be a good (or bad?) idea to only consider a smaller parts of the image (eg. only the parts near the eye?) -- some parts of the image may be irrelevant.
7. Sometimes it can be a good idea to randomly add noise, or randomly add some variations to the images. (keyword: data-augmentation)
8. It can be a good idea to quickly explore different approaches on a subset of the dataset, and maybe at a lower resolution (to make the algorithm converge faster)
9. The dataset is imbalanced -- there are 78% of "Young" people in the dataset. It is sometimes a good idea to consider a balanced subset of the dataset to fit your algorithm.
10. Makes absolutely sure that you do not use the test dataset (i.e. the last 5,000 images) to train your algorithm.

### Last Remark:
It is absolutely crucial that your report is readable and well presented. The point of this assignment is not to show me that you know how to re-implement all the things we've done in class. The point of this assignment is to get the job done, as simply, robustly and efficiently as possible.

# Work starts here

## Creating balanced preliminary training & test data

In [ ]:
# select first 15000 rows for training dataset
train = attribute[:15000]

# create id column
id = list(range(1,15001))
train["id"] = id

In [ ]:
is_Young = train["Young"]==1
is_Old = train["Young"]==-1

# subset training set by young/old
train_y = train[is_Young] 
train_o = train[is_Old]

# select indices for prelim training set
onp.random.seed(0)
y_pindex = onp.random.choice(len(train_y), 2000, replace=False)
o_pindex = onp.random.choice(len(train_o), 2000, replace=False)

# select prelim training data
ptrain_y = train_y.iloc[y_pindex[:1500]] # need to test if this works
ptest_y = train_y.iloc[y_pindex[1500:]]
ptrain_o = train_o.iloc[o_pindex[:1500]]
ptest_o = train_o.iloc[o_pindex[1500:]]

In [ ]:
# collate prelim training data (3k obs, 1.5k each class)
ptrain = ptrain_y.append(ptrain_o)
ptrain = ptrain.sort_values('id')

# collate prelim test data (1k obs, .5k each class)
ptest = ptest_y.append(ptest_o)
ptest = ptest.sort_values('id')

In [ ]:
# list of all indices selected for prelim training data
fn_ptrain = ptrain['id'].values.tolist()

# list of all indices selected for prelim test data
fn_ptest = ptest['id'].values.tolist()

In [ ]:
# load the images from the prelim training set
ntrain_img = 3000
ptrain_img = onp.zeros((ntrain_img,100,100))
for k in range(ntrain_img):
    im = imageio.imread(os.path.join(path, all_img[fn_ptrain[k]])).astype(float)
    im = resize(im, (100,100))
    im = onp.mean(im, axis=2)
    ptrain_img[k,:,:] = im
    
# load the images from the prelim training set
ntest_img = 1000
ptest_img = onp.zeros((ntest_img,100,100))
for k in range(ntest_img):
    im = imageio.imread(os.path.join(path, all_img[fn_ptest[k]])).astype(float)
    im = resize(im, (100,100))
    im = onp.mean(im, axis=2)
    ptest_img[k,:,:] = im

From code above, `ptrain` is the .csv equivalent for the preliminary TRAINING data while `ptrain_img` are the relevant images for the preliminary training data. `ptest` is the .csv equivalent for the preliminary TEST data while `ptest_img` are the relevant images for the preliminary training data. 

In [ ]:
data_train = ptrain_img.reshape(3000,100*100)/255.
y_train = onp.array(ptrain['Young']).astype(float)
data_val = ptest_img.reshape(1000,100*100)/255.
y_val = onp.array(ptest['Young']).astype(float)

# Optimisation starts here

Logistic regression loss function:
$$ \frac{1}{N} \, \sum_{i=1}^N \log(1 + \exp(-y_i \, \langle \beta, x_i \rangle)) $$

Logistic probability:
$$P(y=1 | x) = \frac{1}{1 + \exp(- \langle \beta, x \rangle)}$$

In [ ]:
def prediction_single(beta, x):
    """ beta is a vector of dimension 10000, and x as well """
    proba = 1. / (1. + np.exp(-np.dot(beta, x)))
    return proba

#make a prediction on a whole dataset
prediction_data = jax.vmap(prediction_single, in_axes=(None,0))

def loss_single(beta,x,y):
    """
    beta: vector of dimension 100*100
    x: vector of dimension 100*100
    y: a number that equals -1 or 1
    """
    return np.log(1. + np.exp(-y * np.dot(beta, x)))

#compute the loss on. whole dataset
loss_dataset = jax.vmap(loss_single, in_axes=(None,0,0))

def loss(beta, data, y):
    """ compute the mean of all the individual losses """
    list_of_all_losses = loss_dataset(beta, data, y)
    return np.mean(list_of_all_losses)

grad_loss = jax.jit(jax.grad(loss))

def accuracy(beta, data, y):
    pred = prediction_data(beta, data)
    threshold = 0.5
    prediction_binary = (pred > 0.5).astype(int)  #equals 1 if pred > threshold and 0 otherwise
    prediction_sign = 2*(prediction_binary - 0.5) #equals 1 if pred > threshold and -1 otherwise
    return np.mean(prediction_sign == y)

In [ ]:
D = 100*100
beta_init = onp.random.normal(0.5, scale = 1./onp.sqrt(D), size=D)

## Gradient Descent

In [ ]:
n_iter = 500
learning_rate = 0.01
loss_history = []
acc_train_list = []
acc_val_list = []


for k in range(n_iter):
    gradient = grad_loss(beta, data_train, y_train)
    beta = beta - learning_rate * gradient
    current_loss = loss(beta, data_train, y_train)
    accuracy_train = accuracy(beta, data_train, y_train)
    accuracy_val = accuracy(beta, data_val, y_val) 
    
    acc_train_list.append(accuracy_train)
    acc_val_list.append(accuracy_val)
    if k % 50 == 0:
        print("Loss:{0:.3f} \t Accuracy: {1:.3f}% / {2:.3f}%".format(current_loss, \
                                             100*accuracy_train,\
                                             100*accuracy_val))
    loss_history.append(current_loss)

## L-BFGS

In [ ]:
def loss_wrapper(beta):
    """ only one argument """
    return loss(beta, data_train, y_train)

grad_loss_wrapper = jax.jit(jax.grad(loss_wrapper))    

def gradient_wrapper(beta):
    """output is a numpy array"""
    return onp.array(grad_loss_wrapper(beta))

In [ ]:
loss_train_history = []  #to save the loss trajectory
loss_val_history = []  #to save the loss trajectory
time_history = []  #to save the compute time

def save_traj(beta):
    """ a function that saves a few statistics for later analysis"""
    loss_train = loss(beta, data_train, y_train)
    loss_val = loss(beta, data_val, y_val)
    timing = time.time() - start
    print("Time:{0:3f} \t Loss(train):{1:.3f} \t Loss(val):{2:.3f}".format(timing, loss_train, loss_val))
    time_history.append(timing)
    loss_train_history.append(loss_train)
    loss_val_history.append(loss_val)

#run LBFGS
start = time.time()
traj = scipy.optimize.minimize(fun = loss_wrapper, #functino to minimize
                                x0 = onp.array(beta_init), #initial guess
                                method='L-BFGS-B',  #we want to use L-BFGS
                                jac=gradient_wrapper, #function that computes the gradient
                                callback=save_traj, #a function used to save some results for later pltting
                                options={"maxiter":100})  #maximum number of iteration

## Stochastic Gradient Descent + Momentum

In [ ]:
n_epoch = 50
batch_size=100  #size of the mini-batch
beta = onp.copy(beta_init)
mov_avg_param = 0.95

learning_rate = 0.1
loss_train_history = []
loss_val_history = []

acc_train_history = []  #computed at the end of each epoch
acc_val_history = []    #computed at the end of each epoch

#initialize the gradient
gradient = grad_loss(beta, data_train, y_train)

start = time.time()
for epoch in range(n_epoch):
    #let us divide the learning by 2 every 10 epoch
    if epoch % 10 == 0 and epoch >= 1:
        learning_rate = learning_rate/2.
        
    for k in range(n_training_data // batch_size):  #number of batch per epoch

        #first option
        #============
        #select at random "batch_size" random data point
        #index_ = onp.random.choice(train_data_size, batch_size, replace=False)

        #second option
        #============
        #consider the batches sequentially
        index_ = onp.arange(k*batch_size, (k+1)*batch_size) % n_training_data
        gradient_local = grad_loss(beta, data_train[index_], y_train[index_])

        #moving average of the gradient
        gradient = gradient*mov_avg_param + (1-mov_avg_param)*gradient_local

        #gradient descent update
        beta = beta - learning_rate * gradient
    
    #accuracy_train = accuracy(beta, data_train, y_train)
    #accuracy_val = accuracy(beta, data_val, y_val) 
    #acc_train_list.append(accuracy_train)
    #acc_val_list.append(accuracy_val)
    
    loss_train = loss(beta, data_train, y_train)
    loss_val = loss(beta, data_val, y_val)
    loss_train_history.append(loss_train)
    loss_val_history.append(loss_val)
    
    timing = time.time() - start
    print("epoch:{0:.0f} \t time:{1:.0f} \t Loss(train):{2:.3f} \t Loss(val):{3:.3f}".format(epoch, 
                                                                                                  timing, 
                                                                                                  loss_train, 
                                                                                                  loss_val))